# Maxwell's Equations: Creating an Einstein Toolkit Thorn

### NRPy+ Source Code for this module: [Maxwell/MaxwellCartesian_Evol.py](../edit/Maxwell/MaxwellCartesian_Evol.py), which is fully documented in the [previous NRPy+ tutorial module](Tutorial-MaxwellCartesian.ipynb) on using NRPy+ to construct Maxwell's equations and initial data as SymPy expressions.

This module focuses on using the equations developed in the [last tutorial](Tutorial-MaxwellCartesian.ipynb) to build an Einstein Toolkit (ETK) thorn to solve Maxwell's equations in Cartesian coordinates. This tutorial will focus on implementing the time evolution aspects; the next will construct the thorn that will set up the initial data to be evolved.

When interfaced properly with the ETK, this module will propagate the initial data for $E_i$, $A_i$, and $\psi$ ( and $\Gamma$, if we so choose), defined in the next tutorial, forward in time by integrating the equations for $\partial_t E_i$, $\partial_t A_i$ and $\partial_t \psi$ (and possibly $\partial_t \Gamma$) subject to spatial boundary conditions. The time evolution itself is handled by the $\text{MoL}$ (Method of Lines) thorn in the $\text{CactusNumerical}$ arrangement, and the boundary conditions by the $\text{Boundary}$ thorn in the $\text{CactusBase}$ arrangement. 

Similar to the Maxwell initial data module, we will construct the WaveToyNRPy module in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the evolution equations into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.


In [1]:
# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop


# Step 1b: This is an Einstein Toolkit (ETK) thorn. Here we
#          tell NRPy+ that gridfunction memory access will 
#          therefore be in the "ETK" style.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")
#Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")
par.set_parval_from_str("outputC::outCverbose",False) # To prevent absurdly large output files.

# Step 1c: Call the MaxwellCartesian_Evol() function from within the
#          Maxwell/MaxwellCartesian_Evol.py module.
import Maxwell.MaxwellCartesian_Evol as mwrhs
mwrhs.MaxwellCartesian_Evol()
# Step 4: Register gridfunctions so they can be written to by NRPy.
# System I:
AIrhsD = ixp.register_gridfunctions_for_single_rank1("EVOL","AIrhsD")
EIrhsD = ixp.register_gridfunctions_for_single_rank1("EVOL","EIrhsD")
psiIrhs = gri.register_gridfunctions("EVOL","psiIrhs")

# Step 5: Set the uu and vv gridfunctions to the uu_Evol & vv_Evol variables 
#         defined by InitialData_PlaneWave().
for i in range(DIM):
    AIrhsD[i] = mwrhs.ArhsD[i]
    EIrhsD[i] = mwrhs.ErhsD[i]
psiIrhs = mwrhs.psi_rhs

# Step 4: Register gridfunctions so they can be written to by NRPy.
# System II:
AIIrhsD = ixp.register_gridfunctions_for_single_rank1("EVOL","AIIrhsD")
EIIrhsD = ixp.register_gridfunctions_for_single_rank1("EVOL","EIIrhsD")
psiIIrhs = gri.register_gridfunctions("EVOL","psiIIrhs")
Gammarhs = gri.register_gridfunctions("EVOL","Gammarhs")

# Step 5: Set the uu and vv gridfunctions to the uu_Evol & vv_Evol variables 
#         defined by InitialData_PlaneWave().
for i in range(DIM):
    AIIrhsD[i] = mwrhs.ArhsD[i]
    EIIrhsD[i] = mwrhs.ErhsD[i]
psiIIrhs = mwrhs.psi_rhs
Gammarhs = mwrhs.Gamma_rhs

# Step 6: Create the C code output kernel.
Maxwell_Evol_to_print = [\
                        lhrh(lhs=gri.gfaccess("out_gfs","AIrhsD0"),rhs=AIrhsD[0]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","AIrhsD1"),rhs=AIrhsD[1]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","AIrhsD2"),rhs=AIrhsD[2]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","EIrhsD0"),rhs=EIrhsD[0]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","EIrhsD1"),rhs=EIrhsD[1]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","EIrhsD2"),rhs=EIrhsD[2]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","psiIrhs"),rhs=psiIrhs),\
                        lhrh(lhs=gri.gfaccess("out_gfs","AIIrhsD0"),rhs=AIIrhsD[0]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","AIIrhsD1"),rhs=AIIrhsD[1]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","AIIrhsD2"),rhs=AIIrhsD[2]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","EIIrhsD0"),rhs=EIIrhsD[0]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","EIIrhsD1"),rhs=EIIrhsD[1]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","EIIrhsD2"),rhs=EIIrhsD[2]),\
                        lhrh(lhs=gri.gfaccess("out_gfs","psiIIrhs"),rhs=psiIIrhs),\
                        lhrh(lhs=gri.gfaccess("out_gfs","Gammarhs"),rhs=Gammarhs),]
Maxwell_Evol_CcodeKernel = fin.FD_outputC("returnstring",Maxwell_Evol_to_print)

Maxwell_Evol_looped = loop.loop(["i2","i1","i0"],["1","1","1"],["cctk_lsh[2]-1","cctk_lsh[1]-1","cctk_lsh[0]-1"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",\
                               Maxwell_Evol_CcodeKernel.replace("time","cctk_time"))

# Step 7: Create directories for the thorn if they don't exist.
!mkdir MaxwellEvol     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir MaxwellEvol/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 8: Write the C code kernel to file.
with open("MaxwellEvol/src/Maxwell_Evol.h", "w") as file:
    file.write(str(Maxwell_Evol_looped))
